In [4]:
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table,Column,MaskedColumn,QTable
from astropy.cosmology import FlatLambdaCDM
import scipy.constants as C
from astropy.coordinates import SkyCoord
from astropy import units as u
import xgboost as xgb
import sklearn
from sklearn.model_selection import KFold, train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import metrics
import sklearn.model_selection
import optuna
import shutil
import os

mpl.rcParams.update({'font.size': 20})
mpl.rcParams['lines.linewidth'] = 1.5
mpl.rcParams['axes.linewidth']=2
font = {'family': 'sans-serif',
        'color':  'black',
        'weight': 'normal',
        'size': 12,
        }

# Version of Python Package

In [5]:
print('Numpy',np.__version__)
print('Pandas',pd.__version__)
print('XGBoost',xgb.__version__)
print('Optuna',optuna.__version__)
print('Scikit-learn',sklearn.__version__)

Numpy 1.21.5
Pandas 1.5.1
XGBoost 1.7.1
Optuna 3.0.3
Scikit-learn 1.0.2


# Read the parent sample

In [21]:
data=ascii.read('./all_info.csv')

# Input features

In [22]:
all_para=pd.DataFrame(
    {
        "fuv_nuv":pd.Series(data['fuv_nuv'],dtype='float'),
        "nuv_u":pd.Series(data['nuv_u'],dtype='float'),
        "nuv_g":pd.Series(data['nuv_g'],dtype='float'),
        "u_g":pd.Series(data['u_g'],dtype='float'),
        "g_r":pd.Series(data['g_r'],dtype='float'),
        "r_i":pd.Series(data['r_i'],dtype='float'),
        "i_z":pd.Series(data['i_z'],dtype='float'),
        "z_y":pd.Series(data['z_y'],dtype='float'),
        "y_J":pd.Series(data['y_J'],dtype='float'),
        "J_H":pd.Series(data['J_H'],dtype='float'),
        "H_K":pd.Series(data['H_K'],dtype='float'),
        "K_3p6":pd.Series(data['K_3p6'],dtype='float'),
        "y_3p6":pd.Series(data['y_3p6'],dtype='float'),
        "i_3p6":pd.Series(data['i_3p6'],dtype='float'),
        "r_3p6":pd.Series(data['r_3p6'],dtype='float'),
        "z_3p6":pd.Series(data['z_3p6'],dtype='float'),
        "g_3p6":pd.Series(data['g_3p6'],dtype='float'),
        "ir3p6_4p5":pd.Series(data['ir3p6_4p5'],dtype='float'),
        "morpho_i":pd.Series(data['morpho_i'],dtype='float'),
        "morpho_r":pd.Series(data['morpho_r'],dtype='float'),
        "morpho_z":pd.Series(data['morpho_z'],dtype='float'),
        "morpho_y":pd.Series(data['morpho_y'],dtype='float'),
        "morpho_g":pd.Series(data['morpho_g'],dtype='float'),
    }
)
all_para=all_para.round(3)


#classifier 1
training_s1=np.where((data['train_test_flag']==1)&\
                    ((data['classlabel_s1']==0)|(data['classlabel_s1']==1)))
training_s1_ext=np.where((data['train_test_flag']==1)&\
                    ((data['classlabel_s1']==1)))
training_s1_star=np.where((data['train_test_flag']==1)&\
                    ((data['classlabel_s1']==0)))
print('N of training sources:',np.size(training_s1),\
     '\nN of training extragalactic objects:',np.size(training_s1_ext),\
     '\nN of training stars:',np.size(training_s1_star))


all_training_para_s1=pd.DataFrame(
    {
        "fuv_nuv":pd.Series(data['fuv_nuv'][training_s1],dtype='float'),
        "nuv_u":pd.Series(data['nuv_u'][training_s1],dtype='float'),
        "nuv_g":pd.Series(data['nuv_g'][training_s1],dtype='float'),
        "u_g":pd.Series(data['u_g'][training_s1],dtype='float'),
        "g_r":pd.Series(data['g_r'][training_s1],dtype='float'),
        "r_i":pd.Series(data['r_i'][training_s1],dtype='float'),
        "i_z":pd.Series(data['i_z'][training_s1],dtype='float'),
        "z_y":pd.Series(data['z_y'][training_s1],dtype='float'),
        "y_J":pd.Series(data['y_J'][training_s1],dtype='float'),
        "J_H":pd.Series(data['J_H'][training_s1],dtype='float'),
        "H_K":pd.Series(data['H_K'][training_s1],dtype='float'),
        "K_3p6":pd.Series(data['K_3p6'][training_s1],dtype='float'),
        "y_3p6":pd.Series(data['y_3p6'][training_s1],dtype='float'),
        "i_3p6":pd.Series(data['i_3p6'][training_s1],dtype='float'),
        "r_3p6":pd.Series(data['r_3p6'][training_s1],dtype='float'),
        "z_3p6":pd.Series(data['z_3p6'][training_s1],dtype='float'),
        "g_3p6":pd.Series(data['g_3p6'][training_s1],dtype='float'),
        "ir3p6_4p5":pd.Series(data['ir3p6_4p5'][training_s1],dtype='float'),
        "morpho_i":pd.Series(data['morpho_i'][training_s1],dtype='float'),
        "morpho_r":pd.Series(data['morpho_r'][training_s1],dtype='float'),
        "morpho_z":pd.Series(data['morpho_z'][training_s1],dtype='float'),
        "morpho_y":pd.Series(data['morpho_y'][training_s1],dtype='float'),
        "morpho_g":pd.Series(data['morpho_g'][training_s1],dtype='float'),
    }
)
all_training_para_s1=all_training_para_s1.round(3)
all_training_label_s1=data['classlabel_s1'][training_s1]



test_s1=np.where((data['train_test_flag']==0)&\
                    ((data['classlabel_s1']==0)|(data['classlabel_s1']==1)))
test_s1_ext=np.where((data['train_test_flag']==0)&\
                    ((data['classlabel_s1']==1)))
test_s1_star=np.where((data['train_test_flag']==0)&\
                    ((data['classlabel_s1']==0)))
print('N of blind-test sources:',np.size(test_s1),\
     '\nN of blind-test extragalactic objects:',np.size(test_s1_ext),\
     '\nN of blind-test stars:',np.size(test_s1_star))
all_test_para_s1=pd.DataFrame(
    {
        "fuv_nuv":pd.Series(data['fuv_nuv'][test_s1],dtype='float'),
        "nuv_u":pd.Series(data['nuv_u'][test_s1],dtype='float'),
        "nuv_g":pd.Series(data['nuv_g'][test_s1],dtype='float'),
        "u_g":pd.Series(data['u_g'][test_s1],dtype='float'),
        "g_r":pd.Series(data['g_r'][test_s1],dtype='float'),
        "r_i":pd.Series(data['r_i'][test_s1],dtype='float'),
        "i_z":pd.Series(data['i_z'][test_s1],dtype='float'),
        "z_y":pd.Series(data['z_y'][test_s1],dtype='float'),
        "y_J":pd.Series(data['y_J'][test_s1],dtype='float'),
        "J_H":pd.Series(data['J_H'][test_s1],dtype='float'),
        "H_K":pd.Series(data['H_K'][test_s1],dtype='float'),
        "K_3p6":pd.Series(data['K_3p6'][test_s1],dtype='float'),
        "y_3p6":pd.Series(data['y_3p6'][test_s1],dtype='float'),
        "i_3p6":pd.Series(data['i_3p6'][test_s1],dtype='float'),
        "r_3p6":pd.Series(data['r_3p6'][test_s1],dtype='float'),
        "z_3p6":pd.Series(data['z_3p6'][test_s1],dtype='float'),
        "g_3p6":pd.Series(data['g_3p6'][test_s1],dtype='float'),
        "ir3p6_4p5":pd.Series(data['ir3p6_4p5'][test_s1],dtype='float'),
        "morpho_i":pd.Series(data['morpho_i'][test_s1],dtype='float'),
        "morpho_r":pd.Series(data['morpho_r'][test_s1],dtype='float'),
        "morpho_z":pd.Series(data['morpho_z'][test_s1],dtype='float'),
        "morpho_y":pd.Series(data['morpho_y'][test_s1],dtype='float'),
        "morpho_g":pd.Series(data['morpho_g'][test_s1],dtype='float'),
    }
)
all_test_para_s1=all_test_para_s1.round(3)
all_test_label_s1=data['classlabel_s1'][test_s1]


#classifier 2
training_s2=np.where((data['train_test_flag']==1)&\
                    ((data['classlabel_s2']==0)|(data['classlabel_s2']==1)))
training_s2_qso=np.where((data['train_test_flag']==1)&\
                    ((data['classlabel_s2']==1)))
training_s2_gal=np.where((data['train_test_flag']==1)&\
                    ((data['classlabel_s2']==0)))
print('N of training sources:',np.size(training_s2),\
     '\nN of training quasars:',np.size(training_s2_qso),\
     '\nN of training galaxies:',np.size(training_s2_gal))

all_training_para_s2=pd.DataFrame(
    {
        "fuv_nuv":pd.Series(data['fuv_nuv'][training_s2],dtype='float'),
        "nuv_u":pd.Series(data['nuv_u'][training_s2],dtype='float'),
        "nuv_g":pd.Series(data['nuv_g'][training_s2],dtype='float'),
        "u_g":pd.Series(data['u_g'][training_s2],dtype='float'),
        "g_r":pd.Series(data['g_r'][training_s2],dtype='float'),
        "r_i":pd.Series(data['r_i'][training_s2],dtype='float'),
        "i_z":pd.Series(data['i_z'][training_s2],dtype='float'),
        "z_y":pd.Series(data['z_y'][training_s2],dtype='float'),
        "y_J":pd.Series(data['y_J'][training_s2],dtype='float'),
        "J_H":pd.Series(data['J_H'][training_s2],dtype='float'),
        "H_K":pd.Series(data['H_K'][training_s2],dtype='float'),
        "K_3p6":pd.Series(data['K_3p6'][training_s2],dtype='float'),
        "y_3p6":pd.Series(data['y_3p6'][training_s2],dtype='float'),
        "i_3p6":pd.Series(data['i_3p6'][training_s2],dtype='float'),
        "r_3p6":pd.Series(data['r_3p6'][training_s2],dtype='float'),
        "z_3p6":pd.Series(data['z_3p6'][training_s2],dtype='float'),
        "g_3p6":pd.Series(data['g_3p6'][training_s2],dtype='float'),
        "ir3p6_4p5":pd.Series(data['ir3p6_4p5'][training_s2],dtype='float'),
        "morpho_i":pd.Series(data['morpho_i'][training_s2],dtype='float'),
        "morpho_r":pd.Series(data['morpho_r'][training_s2],dtype='float'),
        "morpho_z":pd.Series(data['morpho_z'][training_s2],dtype='float'),
        "morpho_y":pd.Series(data['morpho_y'][training_s2],dtype='float'),
        "morpho_g":pd.Series(data['morpho_g'][training_s2],dtype='float'),
    }
)
all_training_para_s2=all_training_para_s2.round(3)
all_training_label_s2=data['classlabel_s2'][training_s2]



test_s2=np.where((data['train_test_flag']==0)&\
                    ((data['classlabel_s2']==0)|(data['classlabel_s2']==1)))
test_s2_qso=np.where((data['train_test_flag']==0)&\
                    ((data['classlabel_s2']==1)))
test_s2_gal=np.where((data['train_test_flag']==0)&\
                    ((data['classlabel_s2']==0)))
print('N of blind-test sources:',np.size(test_s2),\
     '\nN of blind-test quasars:',np.size(test_s2_qso),\
     '\nN of blind-test galaxies:',np.size(test_s2_gal))

all_test_para_s2=pd.DataFrame(
    {
        "fuv_nuv":pd.Series(data['fuv_nuv'][test_s2],dtype='float'),
        "nuv_u":pd.Series(data['nuv_u'][test_s2],dtype='float'),
        "nuv_g":pd.Series(data['nuv_g'][test_s2],dtype='float'),
        "u_g":pd.Series(data['u_g'][test_s2],dtype='float'),
        "g_r":pd.Series(data['g_r'][test_s2],dtype='float'),
        "r_i":pd.Series(data['r_i'][test_s2],dtype='float'),
        "i_z":pd.Series(data['i_z'][test_s2],dtype='float'),
        "z_y":pd.Series(data['z_y'][test_s2],dtype='float'),
        "y_J":pd.Series(data['y_J'][test_s2],dtype='float'),
        "J_H":pd.Series(data['J_H'][test_s2],dtype='float'),
        "H_K":pd.Series(data['H_K'][test_s2],dtype='float'),
        "K_3p6":pd.Series(data['K_3p6'][test_s2],dtype='float'),
        "y_3p6":pd.Series(data['y_3p6'][test_s2],dtype='float'),
        "i_3p6":pd.Series(data['i_3p6'][test_s2],dtype='float'),
        "r_3p6":pd.Series(data['r_3p6'][test_s2],dtype='float'),
        "z_3p6":pd.Series(data['z_3p6'][test_s2],dtype='float'),
        "g_3p6":pd.Series(data['g_3p6'][test_s2],dtype='float'),
        "ir3p6_4p5":pd.Series(data['ir3p6_4p5'][test_s2],dtype='float'),
        "morpho_i":pd.Series(data['morpho_i'][test_s2],dtype='float'),
        "morpho_r":pd.Series(data['morpho_r'][test_s2],dtype='float'),
        "morpho_z":pd.Series(data['morpho_z'][test_s2],dtype='float'),
        "morpho_y":pd.Series(data['morpho_y'][test_s2],dtype='float'),
        "morpho_g":pd.Series(data['morpho_g'][test_s2],dtype='float'),
    }
)
all_test_para_s2=all_test_para_s2.round(3)
all_test_label_s2=data['classlabel_s2'][test_s2]

N of training sources: 3710 
N of training extragalactic objects: 3396 
N of training stars: 314
N of blind-test sources: 2051 
N of blind-test extragalactic objects: 1896 
N of blind-test stars: 155
N of training sources: 3396 
N of training quasars: 723 
N of training galaxies: 2673
N of blind-test sources: 1896 
N of blind-test quasars: 393 
N of blind-test galaxies: 1503


# Use Optuna to Find Optimal Hyperparameters for Classifier 1

In [ ]:
SEED = 100
N_FOLDS = 5
CV_RESULT_DIR = "./xgboost_cv_results_clf1"    
dtrain_s1=xgb.DMatrix(all_training_para_s1,label=all_training_label_s1,missing=999.000)

def objective(trial):
    param={
            "objective":"binary:logistic",
            "booster":"gbtree",
            "tree_method":"hist",
            "eval_metric":"logloss",
            "reg_lambda":trial.suggest_float("reg_lambda",0.1,10,step=0.1),
            "reg_alpha":trial.suggest_float("reg_alpha",0,10,step=0.1),
            "max_depth":trial.suggest_int("max_depth",5,10),
            "eta":trial.suggest_float("eta",0.1,0.3,step=0.1),
            "gamma":trial.suggest_float("gamma",0.1,1,step=0.1),
            "grow_policy":"depthwise",
            "min_child_weight":trial.suggest_int("min_child_weight",1,10,step=1),
            "colsample_bytree":1,
            "subsample":1,
            "max_delta_step":trial.suggest_float("max_delta_step",0,10,step=1),
            }

    xgb_cv_results = xgb.cv(
        params=param,
        dtrain=dtrain_s1,
        num_boost_round=100,
        nfold=N_FOLDS,
        stratified=True,
        seed=SEED,
        verbose_eval=False,
    )
    # Save cross-validation results.
    filepath = os.path.join(CV_RESULT_DIR, "{}.csv".format(trial.number))
    xgb_cv_results.to_csv(filepath, index=False)
    # Extract the best score.
    logloss = xgb_cv_results["test-logloss-mean"].values[-1]
    return logloss

if __name__ == "__main__":
    if not os.path.exists(CV_RESULT_DIR):
        os.mkdir(CV_RESULT_DIR)
    study_step1=optuna.create_study(direction="minimize")
    study_step1.optimize(objective,n_trials=5000)
    print("Number of finished trials: ", len(study_step1.trials))
    #write down the best hyperparameter set
    f=open('step1_best_param.txt',"a+")
    f.write("Best trial:")
    btrial=study_step1.best_trial
    f.write("  Value: {}".format(btrial.value)+"\n")
        
    param_step1={"objective":"binary:logistic",
                     "booster":"gbtree",
                     "tree_method":"hist",
                     "missing":999.000,
                     "eval_metric":"logloss",
                     "n_estimators":100,
                     "reg_lambda":np.around(study_step1.best_trial.params['reg_lambda'],decimals=1),
                     "reg_alpha":np.around(study_step1.best_trial.params['reg_alpha'],decimals=1),
                     "max_depth":study_step1.best_trial.params['max_depth'],
                     "learning_rate":np.around(study_step1.best_trial.params['eta'],decimals=1),
                     "gamma":np.around(study_step1.best_trial.params['gamma'],decimals=1),
                     "grow_policy":"depthwise",
                     "min_child_weight":np.around(study_step1.best_trial.params['min_child_weight'],decimals=1),
                     "colsample_bytree":1,
                     "subsample":1,
                     "max_delta_step":study_step1.best_trial.params['max_delta_step']
                    }
           
    f.write(str(param_step1)+'\n')
    f.close()
    shutil.rmtree(CV_RESULT_DIR)

# First Classifier: Extragalactic Objects versus Stars

In [23]:
#optimal hyperparameters for classifier 1
param_step1={'objective': 'binary:logistic', 'booster': 'gbtree', 'tree_method': 'hist', 'eval_metric': 'logloss', 'reg_lambda': 0.4, 'reg_alpha': 0.2, 'max_depth': 5, 'learning_rate': 0.1, 'gamma': 0.4, 'grow_policy': 'depthwise', 'min_child_weight': 1, 'colsample_bytree': 1, 'subsample': 1, 'max_delta_step': 10.0}

#XGBoost 5-fold cross-validation
SEED = 100
N_FOLDS = 5
dtrain_s1=xgb.DMatrix(all_training_para_s1,label=all_training_label_s1,missing=999.000)
xgb_cv_results = xgb.cv(
    params=param_step1,
    dtrain=dtrain_s1,
    num_boost_round=100,
    nfold=N_FOLDS,
    stratified=True,
    seed=SEED,
    verbose_eval=False,
)
print('Mean logloss value of five-fold cross-validation:',\
      xgb_cv_results["test-logloss-mean"].values[-1])


#construct Classifier 1
param_step1={'objective': 'binary:logistic', 'booster': 'gbtree', 'tree_method': 'hist', 'missing': 999.0, 'eval_metric': 'logloss', 'n_estimators': 100, 'reg_lambda': 0.4, 'reg_alpha': 0.2, 'max_depth': 5, 'learning_rate': 0.1, 'gamma': 0.4, 'grow_policy': 'depthwise', 'min_child_weight': 1, 'colsample_bytree': 1, 'subsample': 1, 'max_delta_step': 10.0}
print('Optimal hyperparameters:',param_step1)
clf1=xgb.XGBClassifier(**param_step1)
training_pred_prob=sklearn.model_selection.cross_val_predict(clf1,all_training_para_s1,all_training_label_s1,cv=5,method='predict_proba')
training_pred_pvalue_star=training_pred_prob[:,0]
training_pred_pvalue_ext=training_pred_prob[:,1]
pred_label_prob_step1_cv=np.zeros(np.size(training_s1))

print('We use p_ext>0.98 as the threshold of classify extragalactic objects.')
pred_label_prob_step1_cv[np.where(training_pred_pvalue_ext>=0.98)]=1
pre_results_step1_cv=metrics.precision_recall_fscore_support(all_training_label_s1,pred_label_prob_step1_cv)
preci_nega=pre_results_step1_cv[0][0]
recal_nega=pre_results_step1_cv[1][0]
    
preci_posi=pre_results_step1_cv[0][1]
recal_posi=pre_results_step1_cv[1][1]

print('Classes:   QSOs/Galaxies (+) Stars(-)  ')
print('5-fold Training Precision+: ',np.around(preci_posi,decimals=4))
print('5-fold Training Recall+: ',np.around(recal_posi,decimals=4))
print('5-fold Training Precision-: ',np.around(preci_nega,decimals=4))
print('5-fold Training Recall-: ',np.around(recal_nega,decimals=4))


Mean logloss value of five-fold cross-validation: 0.01041666195816765
Optimal hyperparameters: {'objective': 'binary:logistic', 'booster': 'gbtree', 'tree_method': 'hist', 'missing': 999.0, 'eval_metric': 'logloss', 'n_estimators': 100, 'reg_lambda': 0.4, 'reg_alpha': 0.2, 'max_depth': 5, 'learning_rate': 0.1, 'gamma': 0.4, 'grow_policy': 'depthwise', 'min_child_weight': 1, 'colsample_bytree': 1, 'subsample': 1, 'max_delta_step': 10.0}
We use p_ext>0.98 as the threshold of classify extragalactic objects.
Classes:   QSOs/Galaxies (+) Stars(-)  
5-fold Training Precision+:  0.9991
5-fold Training Recall+:  0.9971
5-fold Training Precision-:  0.9688
5-fold Training Recall-:  0.9904


# Use Optuna to Find Optimal Hyperparameters for Classifier 2

In [ ]:
SEED = 100
N_FOLDS = 5
CV_RESULT_DIR = "./xgboost_cv_results_clf2"
dtrain_s2=xgb.DMatrix(all_training_para_s2,label=all_training_label_s2,missing=999.000)


def objective(trial):
    param={
            "objective":"binary:logistic",
            "booster":"gbtree",
            "tree_method":"hist",
            "eval_metric":"logloss",
            "reg_lambda":trial.suggest_float("reg_lambda",0.1,10,step=0.1),
            "reg_alpha":trial.suggest_float("reg_alpha",0,10,step=0.1),
            "max_depth":trial.suggest_int("max_depth",5,10),
            "eta":trial.suggest_float("eta",0.1,0.3,step=0.1),
            "gamma":trial.suggest_float("gamma",0.1,10,step=0.1),
            "grow_policy":"depthwise",
            "min_child_weight":trial.suggest_int("min_child_weight",1,10,step=1),
            "colsample_bytree":1,
            "subsample":1,
            "max_delta_step":trial.suggest_float("max_delta_step",0,10,step=1),
            }
    xgb_cv_results=xgb.cv(
        params=param,
        dtrain=dtrain_s2,
        num_boost_round=100,
        nfold=N_FOLDS,
        stratified=True,
        seed=SEED,
        verbose_eval=False,
    )

    # Save cross-validation results.
    filepath = os.path.join(CV_RESULT_DIR, "{}.csv".format(trial.number))
    xgb_cv_results.to_csv(filepath, index=False)
    # Extract the best score.
    logloss = xgb_cv_results["test-logloss-mean"].values[-1]
    return logloss

if __name__ == "__main__":
    if not os.path.exists(CV_RESULT_DIR):
        os.mkdir(CV_RESULT_DIR)
    study_step2=optuna.create_study(direction="minimize")
    study_step2.optimize(objective,n_trials=5000)
    #write down the best hyperparameter set
    f=open('step2_best_param.txt',"a+")
    f.write("Best trial:")
    btrial=study_step2.best_trial
    f.write("  Value: {}".format(btrial.value)+"\n")
        
    param_step2={"objective":"binary:logistic",
                 "booster":"gbtree",
                 "tree_method":"hist",
                 "missing":999.000,
                 "eval_metric":"logloss",
                 "n_estimators":100,
                 "reg_lambda":np.around(study_step2.best_trial.params['reg_lambda'],decimals=1),
                 "reg_alpha":np.around(study_step2.best_trial.params['reg_alpha'],decimals=1),
                 "max_depth":study_step2.best_trial.params['max_depth'],
                 "learning_rate":np.around(study_step2.best_trial.params['eta'],decimals=1),
                 "gamma":np.around(study_step2.best_trial.params['gamma'],decimals=1),
                 "grow_policy":"depthwise",
                 "min_child_weight":np.around(study_step2.best_trial.params['min_child_weight'],decimals=1),
                 "colsample_bytree":1,
                 "subsample":1,
                 "max_delta_step":study_step2.best_trial.params['max_delta_step']
                    }
           
    f.write(str(param_step2)+'\n')
    f.close()
    shutil.rmtree(CV_RESULT_DIR)

# Second Classifier: Quasars versus Galaxies

In [24]:
#optimal hyperparameters for classifier 2
param_step2={'objective': 'binary:logistic', 'booster': 'gbtree', 'tree_method': 'hist', 'eval_metric': 'logloss', 'reg_lambda': 7.0, 'reg_alpha': 0.5, 'max_depth': 10, 'learning_rate': 0.3, 'gamma': 0.1, 'grow_policy': 'depthwise', 'min_child_weight': 3, 'colsample_bytree': 1, 'subsample': 1, 'max_delta_step': 7.0}

#XGBoost 5-fold cross-validation
SEED = 100
N_FOLDS = 5
dtrain_s2=xgb.DMatrix(all_training_para_s2,label=all_training_label_s2,missing=999.000)
xgb_cv_results = xgb.cv(
    params=param_step2,
    dtrain=dtrain_s2,
    num_boost_round=100,
    nfold=N_FOLDS,
    stratified=True,
    seed=SEED,
    verbose_eval=False,
)
print('Mean logloss value of five-fold cross-validation:',\
      xgb_cv_results["test-logloss-mean"].values[-1])
param_step2={'objective': 'binary:logistic', 'booster': 'gbtree', 'tree_method': 'hist', 'missing': 999.0, 'eval_metric': 'logloss', 'n_estimators': 100, 'reg_lambda': 7.0, 'reg_alpha': 0.5, 'max_depth': 10, 'learning_rate': 0.3, 'gamma': 0.1, 'grow_policy': 'depthwise', 'min_child_weight': 3, 'colsample_bytree': 1, 'subsample': 1, 'max_delta_step': 7.0}
print(param_step2)
clf2=xgb.XGBClassifier(**param_step2)

training_pred_prob=sklearn.model_selection.cross_val_predict(clf2,all_training_para_s2,all_training_label_s2,cv=5,method='predict_proba')
training_pred_pvalue_gal=training_pred_prob[:,0]
training_pred_pvalue_qso=training_pred_prob[:,1]
pred_label_prob_step2_cv=np.zeros(np.size(training_s2))

print('We use p_ext>0.95 as the threshold of classify quasars.')
pred_label_prob_step2_cv[np.where(training_pred_pvalue_qso>=0.95)]=1
pre_results_step2_cv=metrics.precision_recall_fscore_support(all_training_label_s2,pred_label_prob_step2_cv)
preci_nega=pre_results_step2_cv[0][0]
recal_nega=pre_results_step2_cv[1][0]
    
preci_posi=pre_results_step2_cv[0][1]
recal_posi=pre_results_step2_cv[1][1]

print('Classes:  QSOs (+) Galaxies(-)')
print('5-fold Training Precision+: ',np.around(preci_posi,decimals=4))
print('5-fold Training Recall+: ',np.around(recal_posi,decimals=4))
print('5-fold Training Precision-: ',np.around(preci_nega,decimals=4))
print('5-fold Training Recall-: ',np.around(recal_nega,decimals=4))

Mean logloss value of five-fold cross-validation: 0.0534823920031788
{'objective': 'binary:logistic', 'booster': 'gbtree', 'tree_method': 'hist', 'missing': 999.0, 'eval_metric': 'logloss', 'n_estimators': 100, 'reg_lambda': 7.0, 'reg_alpha': 0.5, 'max_depth': 10, 'learning_rate': 0.3, 'gamma': 0.1, 'grow_policy': 'depthwise', 'min_child_weight': 3, 'colsample_bytree': 1, 'subsample': 1, 'max_delta_step': 7.0}
We use p_ext>0.95 as the threshold of classify quasars.
Classes:  QSOs (+) Galaxies(-)
5-fold Training Precision+:  0.9888
5-fold Training Recall+:  0.8534
5-fold Training Precision-:  0.9618
5-fold Training Recall-:  0.9974


# Apply Classifier 1 & 2 to Parent Sample and Evaluating Performance from the Results of Blind-test Sample

In [29]:
# calculate the probability of each source in parent sample
param_step1={'objective': 'binary:logistic', 'booster': 'gbtree', 'tree_method': 'hist', 'missing': 999.0, 'eval_metric': 'logloss', 'n_estimators': 100, 'reg_lambda': 0.4, 'reg_alpha': 0.2, 'max_depth': 5, 'learning_rate': 0.1, 'gamma': 0.4, 'grow_policy': 'depthwise', 'min_child_weight': 1, 'colsample_bytree': 1, 'subsample': 1, 'max_delta_step': 10.0}
clf1=xgb.XGBClassifier(**param_step1)
clf1.fit(all_training_para_s1,all_training_label_s1)

pred_step1_pvalue=clf1.predict_proba(all_para)
pred_pvalue_sta=pred_step1_pvalue[:,0]
pred_pvalue_ext=pred_step1_pvalue[:,1]

param_step2={'objective': 'binary:logistic', 'booster': 'gbtree', 'tree_method': 'hist', 'missing': 999.0, 'eval_metric': 'logloss', 'n_estimators': 100, 'reg_lambda': 7.0, 'reg_alpha': 0.5, 'max_depth': 10, 'learning_rate': 0.3, 'gamma': 0.1, 'grow_policy': 'depthwise', 'min_child_weight': 3, 'colsample_bytree': 1, 'subsample': 1, 'max_delta_step': 7.0}
clf2=xgb.XGBClassifier(**param_step2)
clf2.fit(all_training_para_s2,all_training_label_s2)

pred_step2_pvalue=clf2.predict_proba(all_para)
pred_pvalue_gal=pred_step2_pvalue[:,0]
pred_pvalue_qso=pred_step2_pvalue[:,1]

thre_ext=0.98
thre_qso=0.95


print('N of training sample quasars as ext:',\
      np.size(np.where((data['train_test_flag']==1)&\
                       (pred_pvalue_ext>thre_ext)&(data['classlabel_s2']==1))))
print('N of training sample quasars as qso:',\
      np.size(np.where((data['train_test_flag']==1)&\
                       (pred_pvalue_ext>thre_ext)&(pred_pvalue_qso>thre_qso)&(data['classlabel_s2']==1))))


print('N of predicted quasars in parent sample:',\
     np.size(pred_qso_all))

#generatet the pvalue file of all predicted quasars
data_output=QTable([data['ra_parent'][pred_qso_all],\
             data['dec_parent'][pred_qso_all],\
             pred_pvalue_sta[pred_qso_all],\
             pred_pvalue_ext[pred_qso_all],\
             pred_pvalue_gal[pred_qso_all],\
             pred_pvalue_qso[pred_qso_all]],\
            names=['ra_parent',\
                   'dec_parent',\
                   'pred_star_prob',\
                   'pred_ext_prob',\
                   'pred_gal_prob',\
                   'pred_qso_prob'])
ascii.write(data_output,'./predqso_all_pvalues.csv',format='csv',overwrite=True)


pred_sta_xmm=np.where((data['train_test_flag']==1)&(pred_pvalue_ext<=thre_ext))
pred_gal_xmm=np.where((data['train_test_flag']==1)&(pred_pvalue_ext>thre_ext)&\
                       (pred_pvalue_qso<thre_qso))
pred_qso_xmm=np.where((data['train_test_flag']==1)&(pred_pvalue_ext>thre_ext)&\
                       (pred_pvalue_qso>thre_qso))
pred_qso_all=np.where((pred_pvalue_ext>thre_ext)&\
                       (pred_pvalue_qso>thre_qso))

print('N of predicted stars, galaxies, and quasars in the XMM-LSS region:',\
      np.size(pred_sta_xmm),\
      np.size(pred_gal_xmm),\
      np.size(pred_qso_xmm),\
     '\nBetter X-ray and multi-wavelength coverage in the XMM-LSS region')


test_qso=np.where((data['train_test_flag']==0)&\
                  (data['classlabel_s2']==1))
test_star=np.where((data['train_test_flag']==0)&\
                  (data['classlabel_s1']==0))
test_gal=np.where((data['train_test_flag']==0)&\
                  (data['classlabel_s2']==0))
print('N of SDSS quasars, stars, and galaxies in the blind-test region:',\
     np.size(test_qso),\
     np.size(test_star),\
     np.size(test_gal))


pred_star_test_true=np.where((data['train_test_flag']==0)&(pred_pvalue_ext<=thre_ext)&\
                           (data['classlabel_s1']==0))
pred_gal_test_true=np.where((data['train_test_flag']==0)&(pred_pvalue_ext>thre_ext)&\
                       (pred_pvalue_qso<=thre_qso)&\
                           (data['classlabel_s2']==0))
pred_qso_test_true=np.where((data['train_test_flag']==0)&(pred_pvalue_ext>thre_ext)&\
                       (pred_pvalue_qso>thre_qso)&\
                           (data['classlabel_s2']==1))

print('N of predicted true SDSS quasars, stars, and galaxies in the blind-test sample:',\
     np.size(pred_qso_test_true),\
     np.size(pred_star_test_true),\
     np.size(pred_gal_test_true))

pred_star_blindtest=np.where((data['train_test_flag']==0)&\
                           (pred_pvalue_ext<=thre_ext)&\
                           ((data['classlabel_s1']==0)|(data['classlabel_s1']==1)))
pred_gal_blindtest=np.where((data['train_test_flag']==0)&\
                           (pred_pvalue_ext>thre_ext)&\
                           (pred_pvalue_qso<=thre_qso)&\
                           ((data['classlabel_s2']==0)|(data['classlabel_s2']==1)|(data['classlabel_s1']==0)))
pred_quasar_blindtest=np.where((data['train_test_flag']==0)&\
                            (pred_pvalue_ext>thre_ext)&\
                            (pred_pvalue_qso>thre_qso)&\
                           ((data['classlabel_s2']==0)|(data['classlabel_s2']==1)))
print('N of predicted blind-test sample SDSS quasars, stars, and galaxies:',\
     np.size(pred_quasar_blindtest),\
     np.size(pred_star_blindtest),\
     np.size(pred_gal_blindtest))

relia_qso=np.around(np.size(pred_qso_test_true)/np.size(pred_quasar_blindtest),decimals=4)
relia_star=np.around(np.size(pred_star_test_true)/np.size(pred_star_blindtest),decimals=4)
relia_gal=np.around(np.size(pred_gal_test_true)/np.size(pred_gal_blindtest),decimals=4)
comple_qso=np.around(np.size(pred_qso_test_true)/np.size(test_qso),decimals=4)
comple_star=np.around(np.size(pred_star_test_true)/np.size(test_star),decimals=4)
cpmple_gal=np.around(np.size(pred_gal_test_true)/np.size(test_gal),decimals=4)
print('Reliability Completeness (blind-test sample)')
print('Quasar:',relia_qso,comple_qso)
print('Galaxy:',relia_gal,cpmple_gal)
print('Star:',relia_star,comple_star)


N of training sample quasars as ext: 723
N of training sample quasars as qso: 651
N of predicted quasars in parent sample: 2784
N of predicted stars, galaxies, and quasars in the XMM-LSS region: 18546 74996 1591 
Better X-ray and multi-wavelength coverage in the XMM-LSS region
N of SDSS quasars, stars, and galaxies in the blind-test region: 393 155 1503
N of predicted true SDSS quasars, stars, and galaxies in the blind-test sample: 344 147 1495
N of predicted blind-test sample SDSS quasars, stars, and galaxies: 345 158 1548
Reliability Completeness (blind-test sample)
Quasar: 0.9971 0.8753
Galaxy: 0.9658 0.9947
Star: 0.9304 0.9484
